# I'm Jay. 

I'm a Developer Advocate at Elastic, based in San Diego, Ca. I enjoy finding unique ways to merge his fascination with productivity, automation, and development to create tools and content to serve the tech  and greater community.

## What I want to talk about is projects that matter

- To you
- To Others
- To People who aren't alive yet

# Did you know?

## If you live in a major metropolitan area in the United States you have access to plenty of data that is available to you.

# That includes <!--INSERT CITY HERE and link to it's Portal --> [San Diego](https://data.sandiego.gov/datasets/)


There is a wide array of data that is available _If you know how to look for it_

<!--The GitHub Repository for your Locations Data Portal and it's catalog -->
- https://github.com/COSD-PANDA/seaboard
- https://github.com/COSD-PANDA/data-inventory/

This catalog spec is defined by the [DCAT-US Schema](https://resources.data.gov/resources/dcat-us/#metadata-file-format---json).

Using the spec I was able to index the data into an Elasticsearch DB

## Here's my code to prep our data...

focus on `modify_dataset`

In [2]:
# dcat_parser.py

"""Load JSON File and Load the Data into Elasticsearch"""

import hashlib
import json
import typing

from elasticsearch.helpers import bulk

def gen_id(*args):
    """Create an unique identifier based on the identifier and contactPoint
    of the dcat data
    """
    return hashlib.sha1(''.join(args).encode("utf-8")).hexdigest()


def modify_dataset(datasets, **kwargs):
    for dataset in datasets:
        dataset['_id'] = gen_id(dataset['identifier'], dataset['contactPoint']['fn'])
        dataset["city"] = kwargs.get('city', '')
        dataset["region"] = kwargs.get('region', '')
        url = dataset['identifier'].replace('_', '-')

        if url.endswith('gis'):
            url = url[:-4]
        dataset['url'] = kwargs.get('base_url') + "/" + url
        yield dataset


def upload(dataset: typing.Sequence, *, client, index, **kwargs):
        bulk(
            client=client,
            index=index,
            actions=modify_dataset(dataset, **kwargs),
        )

## Now Let's Load our Data

In [1]:
# upload.py
from dotenv import load_dotenv
import os
import json
from dcat_parser import upload
from connection import client

load_dotenv('.env')

es_index = os.environ.get("ES_INDEX")
input_file = "data.json"

with open(input_file) as json_file:
    upload(
            json.load(json_file)["dataset"],
            client=client,
            index = es_index,
            city="San Diego",
            region="California",
            base_url="https://data.sandiego.gov/datasets/"
            )

You can help me with this if you like on GitHub.
- [DcatParser](https://github.com/kjaymiller/elasticsearch_dcat_us_parser/)
- [OpenCities]()